In [2]:
import pandas as pd

In [68]:
file_txt = open('wr112570.txt', 'r')
file_csv = open('dataset.csv', 'w')
for line in file_txt:
    stroka = line.replace(';', ',').replace(' ', '')
    file_csv.write(stroka)

file_csv.close()
file_txt.close()

'''Создаем файл в формате csv'''

In [67]:
columns = ['station', 'year', 'month', 'day', 'summary', 'min_temperature', 'avarage_temperature', 'max_temperature', 'precipitation']
df = pd.read_csv('dataset.csv', names = columns)

'''Создаем датафрейм'''


In [66]:
df_filtered = df[(df['month'] > 4) & (df['month'] < 10)]

'''Оставляем только нужные месяцы: май, июнь, июль, август, сентябрь'''

In [65]:
df_cleaned = df_filtered.drop(columns=['summary', 'min_temperature', 'precipitation'], axis=1)

df_cleaned.dropna(inplace=True)
df_cleaned['station'] = df_cleaned['station'].astype(int)

df_cleaned.to_csv('cleaned_dataset.csv',  index=False)

'''Удаляем ненужные столбцы и строки с пустями значениями, записываем в новый файл '''

In [69]:
df_first = pd.read_csv('cleaned_dataset.csv')
df_second = pd.read_csv('stations.csv')
df_new = df_first.merge(df_second, how='left', on='station')

'''Добавляем данные о регионе метеостанаций'''

In [63]:
df_new.dropna(axis=0, how="any", thresh=None, subset=None, inplace=True)
df_new['number of region'] = df_new['number of region'].astype(int)

In [8]:
"""Считаем климатическую норму для периода 1960-2009"""

norma = round(df_new[(df_new['year'] > 1959) & (df_new['year'] < 2010)].groupby(['day','month','station'])['max_temperature'].mean(), 1)
norma
df_norma = pd.DataFrame(norma, columns=['max_temperature']).reset_index()
df_norma.rename(columns={'max_temperature':'climatic_norm'}, inplace=True)
df_climatic_norm = df_new.merge(df_norma, how='outer', on=['day', 'month','station'])

In [70]:
'''Смотрим разницу среднесуточных температур и климатической нормы в период с 2010 по 2019 гг.'''

df_result = df_climatic_norm[(df_climatic_norm['year'] > 2009) & (df_climatic_norm['year'] < 2020)]
df_result['difference'] = round(df_climatic_norm['max_temperature'] - df_climatic_norm['climatic_norm'],1)
df_result.to_csv('anomalies.csv',  index=False)

In [59]:
'''Считаем среднемесячное температурное отклонение для всех регионов России в период с 2010 по 2019 гг.'''

df_month = pd.DataFrame(df_result.groupby(['region','station','year','month']).sum()['difference']).reset_index()

def average(row):
    if row.month == 5 or row.month == 7 or row.month == 8:
        month_average = row.difference / 31
    else:
        month_average = row.difference / 30
    return month_average

df_month['month_average'] = df_month.apply(lambda row: average(row), axis=1)


df_region_average_station = pd.DataFrame(df_month.groupby(['region', 'year', 'month']).count()['station'])
df_region_average_month = pd.DataFrame(df_month.groupby(['region', 'year', 'month']).sum()['month_average'])

df_united = df_region_average_station.merge(df_region_average_month, how='outer', on=['region', 'year', 'month'])
df_united['mean_monthly_temperature_deviation'] = round(df_united['month_average'] / df_united['station'], 2)
df_result_mean = df_united.drop(columns=['station', 'month_average'], axis=1).reset_index()
df_result_mean.to_excel('mean_monthly_temperature_deviation.xlsx',  index=True)
df_result_mean.to_csv('mean_monthly_temperature_deviation.csv',  index=True)